In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt
from sklearn.metrics import f1_score,confusion_matrix

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
import pickle

with open("./pickle/token_data.pickle", "rb") as f:
    token_data = pickle.load(f)

In [5]:
comment = token_data["comment"]
target = token_data["target"]
reviews_idx = token_data["comment_ix"]
word2idx = token_data["word2ix"]
idx2word = token_data["ix2word"]
max_seq_length = token_data["max_seq_length"]
idx2target={1:'highlight',0:'no'}

In [6]:
def padding(token):
    token_post=token.copy()
    len_sentence=[ len(i) for i in token_post]
    max_sentence=max(len_sentence) #최대 문장길이 9591
    n=-1
    for i in token_post:
        n+=1
        k=len(token_post[n])
        while max_sentence-k>=0:
            token_post[n].append('<PAD>')
            k+=1
    return token_post

In [7]:
comment=list(map(lambda x: padding(x),  [comment]))
comment=comment[0]

In [8]:
li1=[]
comment_list=[]
for i in comment:
    for k in i:
        li1.append(word2idx[k])
    
    comment_list.append(li1)
    li1=[]

In [9]:
X_train,X_test,y_train,y_test=train_test_split(comment_list,target,test_size=0.3,random_state=7)

In [10]:
from gensim.models import Word2Vec
model = Word2Vec.load('word2vec_50_ver2.model')

C:\Users\jimmi\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [11]:
import numpy as np
import torch

embedding_matrix = []

for word in word2idx.keys():
    try:
        embedding_matrix.append(model[word])
    except:
        embedding_matrix.append(np.zeros(50))

print(len(word2idx))
print(len(embedding_matrix))

embedding_matrix = torch.Tensor(embedding_matrix)

C:\Users\jimmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


17786
17786


In [12]:
embedding_matrix

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.9961,  0.4729,  0.2791,  ...,  0.2730, -1.1971,  0.1223],
        ...,
        [-0.0096,  0.0076, -0.0067,  ..., -0.0008, -0.0002,  0.0026],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0720,  0.5453, -0.8303,  ..., -0.7007, -0.9369, -0.6074]])

In [13]:
class BoW_Clf_Embed_mean(nn.Module):
    def __init__(self, vocab_size, embedding_size, output_size, embedding_matrix):
        super(BoW_Clf_Embed_mean, self).__init__()
#         self.embed = nn.Embedding.from_pretrained(embedding_matrix, freeze=False) # True is Default
        self.embed = nn.Embedding.from_pretrained(embedding_matrix,freeze=False)
        
        self.rnn=nn.RNN(batch_first=True,)
        
        self.linear = nn.Linear(embedding_size, output_size)
    
    def forward(self,inputs):
        embed = self.embed(inputs)
        embed_mean = torch.mean(embed, 1)
        return self.linear(embed_mean)

In [14]:
EPOCHS = 100
LR = 0.1

model = BoW_Clf_Embed_mean(len(word2idx),50, 2, embedding_matrix).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=LR, momentum=0.9)

for epoch in range(EPOCHS):

    model.zero_grad()
    inputs = Variable(torch.LongTensor(X_train)).to(device)
    targets = Variable(torch.LongTensor(y_train)).to(device)
    
    preds = model(inputs)
    
    loss = loss_function(preds, targets)

    if epoch % 10 == 0:
        print(loss.item())
    
    loss.backward()
    optimizer.step()

0.6809141039848328
0.5512250661849976
0.5296952128410339
0.5332592725753784
0.5291300415992737
0.528171181678772
0.5281693339347839
0.5278354287147522
0.5276939272880554
0.527576744556427


In [15]:
correct = 0
list_predicted=[]
model.eval()
for i, seq in enumerate(X_test):
    input = Variable(torch.LongTensor(seq).view(1,-1)).to(device)
    pred = model(input)
    _, pred = torch.max(pred, 1)
    list_predicted.append(pred)
    true = y_test[i]
    if true == pred.item():
        correct +=1
    
    if i%10000 == 0:
        input_seq = [idx2word.get(ix) for ix in seq if ix != 0]
        print("Input :", input_seq)
        print("Prediction :", pred.item())
        print("Truth :", y_test[i])
        print("\n")

print("Accuracy :%s" %(correct/len(X_test)*100))
print('F1 score:%s' %(f1_score(list_predicted,y_test)))

Input : ['킹/Noun', '리/Noun', '아/Exclamation', '나/Noun']
Prediction : 0
Truth : 0


Input : ['쉔무/Noun', '새/Noun']
Prediction : 0
Truth : 0


Accuracy :77.55161809928246


C:\Users\jimmi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


F1 score:0.0


In [16]:
print(confusion_matrix(list_predicted,y_test))

[[10592  3066]
 [    0     0]]
